In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
cols_list = [item for item in range(2,35,1)]
schedule = pd.read_excel('./Data/MLB_schedule_2020.xls', header = 1, usecols = cols_list)


In [3]:
schedule = schedule.rename(columns ={'Ana':'LAA', 'Fla': 'Mia', 'Los': 'LAD', 'Sfo': 'Sanf', 'Sdg': 'Sand'})

In [4]:
team_name_values = sorted(list(schedule.columns))

In [5]:
URL = 'https://blogs.fangraphs.com/how-much-do-the-playoff-odds-change-in-a-shorter-season/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [6]:
results = soup.find_all('table')

In [7]:
df_162 = pd.read_html(str(results))[7]


In [8]:
df_162

,Team,W,L,GB,PCT,Div%,WC%,Playoff%,WS Win%
0,New York Yankees,96,66,—,.593,61.3%,29.2%,90.5%,12.7%
1,Tampa Bay Rays,92,70,4,.568,32.6%,44.6%,77.2%,7.8%
2,Boston Red Sox,85,77,11,.525,6.0%,25.9%,31.9%,2.0%
3,Toronto Blue Jays,73,89,23,.451,0.0%,0.8%,0.9%,0.0%
4,Baltimore Orioles,57,105,39,.352,0.0%,0.0%,0.0%,0.0%
5,Team,W,L,GB,PCT,Div%,WC%,Playoff%,WS Win%
6,Minnesota Twins,91,71,—,.562,60.9%,14.5%,75.4%,8.5%
7,Cleveland Indians,88,74,3,.543,30.3%,20.9%,51.2%,4.4%
8,Chicago White Sox,82,80,9,.506,8.7%,10.0%,18.7%,1.3%
9,Kansas City Royals,71,91,20,.438,0.2%,0.2%,0.3%,0.0%


In [9]:
df_140 = pd.read_html(str(results))[8]

In [10]:
df_110 = pd.read_html(str(results))[9]

In [11]:
df_81 = pd.read_html(str(results))[10]

In [12]:
def win_percentage_df(df, team_name_values, schedule, games_to_be_played):
    game_start_point = 162 - games_to_be_played
    rows_to_drop = [5,11,17,23,29]
    df = df.drop(rows_to_drop)
    df = df[['Team', 'PCT']]
    df = df.reset_index(drop = True)
    team_name_keys = sorted(list(df['Team'].values))
    team_dict = dict(zip(team_name_keys, team_name_values))
    df['Team'] = df['Team'].map(team_dict)
    win_pct_dict = dict(zip(df['Team'], df['PCT']))
    team_columns = list(schedule.columns)
    schedule_list = []
    road_games_list = []
    home_games_list = []
    for team in schedule:
        schedule[team] = schedule[team].str.replace('@','')
        schedule[team] = schedule[team].replace(to_replace = ['Ana', 'Fla', 'Los', 'Sfo', 'Sdg'], value = ['LAA', 'Mia', 'LAD', 'Sanf', 'Sand'])
        schedule[team] = schedule[team].replace(win_pct_dict)
        team_schedule = list(schedule[team].dropna())
        schedule_list.append(team_schedule)
    final_df = pd.DataFrame(schedule_list)
    final_df = final_df.T
    final_df.columns = team_columns
    final_df = final_df[game_start_point:].reset_index(drop = True)
    return final_df
    

In [41]:
def number_road_games(schedule, games_to_be_played):
    df = schedule.copy()
    game_start_point = 162 - games_to_be_played 
    teams = list(schedule.columns)
    road_games_list = []
    for team in df:
        df[team] = df[team].dropna().reset_index(drop = True)
        df[team] = df[team][game_start_point:]
        road_games = df[team].str.count('@').sum()
        road_games_list.append(road_games)
    combined_list = list(zip(teams, road_games_list))
    road_games_df = pd.DataFrame(combined_list, columns = ['Teams', 'Road Games']).set_index('Teams')
    final_df = road_games_df.copy()
    final_df['Home Games'] = games_to_be_played - final_df['Road Games']
    return final_df

In [48]:
home_road_games_df_81 = number_road_games(schedule, 81)

In [49]:
home_road_games_df_81

,Road Games,Home Games
Teams,,
LAA,40.0,41.0
Bal,40.0,41.0
Bos,37.0,44.0
ChW,40.0,41.0
Cle,41.0,40.0
Det,42.0,39.0
Hou,44.0,37.0
Kan,37.0,44.0
Min,42.0,39.0


In [50]:
home_road_games_df_81.to_csv('./Data/home_road_games_81.csv')